In [1]:
import pandas
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import os

In [2]:
folders = [
    'data/upload',
    'models',
    'trainers',
    'infernce',
    'ui'
]

for f in folders:
  os.makedirs(f, exist_ok=True)
  print(f'Folders created {f}')

Folders created data/upload
Folders created models
Folders created trainers
Folders created infernce
Folders created ui


In [3]:
from google.colab import files

In [4]:
from io import BytesIO
def upload_class_image(class_name):
  print(f'upload images of {class_name}')
  uploaded_images = files.upload()
  class_path = f'data/upload/{class_name}'
  os.makedirs(class_path,exist_ok=True)
  for fname, fbytes in uploaded_images.items():
    img = Image.open(BytesIO(fbytes)).convert('RGB')
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.save(os.path.join(class_path,fname))
  print(f'saved {len(uploaded_images)} images in class {class_name}')

In [5]:
upload_class_image('pandas')

upload images of pandas


Saving download (9).jpeg to download (9) (2).jpeg
Saving download (8).jpeg to download (8) (2).jpeg
Saving download (7).jpeg to download (7) (2).jpeg
Saving 254.jpeg to 254 (2).jpeg
Saving pandas1.jpeg to pandas1 (2).jpeg
saved 5 images in class pandas


In [6]:
upload_class_image('bear')

upload images of bear


Saving images (2).jpeg to images (2) (2).jpeg
Saving images (1).jpeg to images (1) (2).jpeg
Saving download (6).jpeg to download (6) (2).jpeg
Saving download (5).jpeg to download (5) (1).jpeg
Saving download (4).jpeg to download (4) (2).jpeg
saved 5 images in class bear


In [7]:
def load_images_labels(base_dir='data/upload'):
  X=[]
  y=[]
  classes = sorted(os.listdir(base_dir))
  for clss in classes:
    clss_path = os.path.join(base_dir,clss)
    if not os.path.isdir(clss_path):
            continue
    for fname in os.listdir(clss_path):
      fpath = os.path.join(clss_path,fname)
      try:
        img = Image.open(fpath).convert('RGB').resize((224,224))
        arr = np.array(img)
        X.append(arr)
        y.append(clss)
      except Exception as e:
        print("Skipping", fpath, ":", e)
  X = np.array(X)
  y = np.array(y)
  print("Loaded:", X.shape, y.shape,  "labels:", np.unique(y))
  return X, y

In [8]:
X, y = load_images_labels()

Loaded: (30, 224, 224, 3) (30,) labels: ['bear' 'pandas']


In [9]:
def flatten_X(X):
    n = X.shape[0]
    return X.reshape(n, -1)

In [10]:
X_flat = flatten_X(X)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def encode_y(y):
    le = LabelEncoder()
    y_encoder = le.fit_transform(y)
    y_categorical = to_categorical(y_encoder)
    return le, y_encoder, y_categorical

In [12]:
le, y_encoder, y_categorical = encode_y(y)

In [13]:
y_encoder.shape

(30,)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X_flat, y_encoder, test_size=.20,random_state=123)

In [15]:
from sklearn.linear_model import LogisticRegression
import joblib

def train_logistic(X_train, y_train, save_path="models/logistic.pkl"):
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    joblib.dump(lr, save_path)
    print("Saved logistic model to", save_path)
    return lr

In [16]:
logistic_model = train_logistic(X_train,y_train)

Saved logistic model to models/logistic.pkl


In [17]:
from sklearn.ensemble import RandomForestClassifier
import joblib

def train_random_forest(X_train, y_train, save_path="models/randomforest.pkl"):
  rf = RandomForestClassifier()
  rf.fit(X_train, y_train)
  joblib.dump(rf,save_path)
  print('saved random forest model to',save_path)
  return rf

In [18]:
random_forest_model = train_random_forest(X_train, y_train)

saved random forest model to models/randomforest.pkl


In [19]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input



def train_cnn(X,y_categorical,epochs=10,save_path="models/cnn.keras", batch_size=32):
    input_shape = X.shape[1:]
    num_classes = y_categorical.shape[1]

    data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    ])

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False


    model = keras.models.Sequential([
        layers.Input(shape=input_shape),

        data_aug,
        #layers.Lambda(preprocess_input),
        base_model,

        layers.GlobalAveragePooling2D(),     # <- FIX (Conv2D ke bajaye GAP)
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')

        #layers.Conv2D(32,(3,3),activation='relu'),
        #layers.MaxPool2D(),
        #layers.Conv2D(64,(3,3),activation='relu'),
        #layers.MaxPool2D(),
        #layers.Conv2D(128,(3,3),activation='relu'),
        #layers.MaxPool2D(),

        #layers.Flatten(),

        #layers.Dense(num_classes,activation='softmax')
      ])

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    checkpoint = ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True, verbose=1)
    early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

    history = model.fit(X, y_categorical, validation_split=0.2, epochs=epochs, batch_size=batch_size,
                        callbacks=[checkpoint, early])

    print("CNN training finished, best model saved to", save_path)
    return model, history


In [20]:
cnn_model, history = train_cnn(X.astype('float32')/255, y_categorical)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.3750 - loss: 0.7216
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to models/cnn.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.3750 - loss: 0.7216 - val_accuracy: 0.0000e+00 - val_loss: 2.6215
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6250 - loss: 1.0320
Epoch 2: val_accuracy did not improve from 0.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.6250 - loss: 1.0320 - val_accuracy: 0.0000e+00 - val_loss: 1.4917
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.6250 - loss: 0.6820
Epoch 3: val_accuracy improved from 0.00000 to 1.00000, saving model to models/cnn.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.6250 - loss: 0.6820 - val_accuracy: 1.0000 - val_loss: 0.4519
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.3750 - loss: 0.6704
Epoch 4: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.37

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def eval_sklearn(model, X_train, y_train, le):
  pred = model.predict(X_train)
  acc = accuracy_score(y_train,pred)
  cm = confusion_matrix(y_train,pred)
  report = classification_report(y_train, pred, target_names=le.classes_)
  print("Accuracy:", acc)
  print("Confusion Matrix:\n", cm)
  print("Classification Report:\n", report)
  return acc, cm, report


def eval_cnn(model, X, y_encoder, le):
  X = X.astype('float32')/255
  preds = model.predict(X)
  preds_idx = np.argmax(preds, axis=1)
  return eval_sklearn_simple(preds_idx, y_encoder, le)

def eval_sklearn_simple(preds_idx, y_encoder, le):
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    acc = accuracy_score(y_encoder, preds_idx)
    cm = confusion_matrix(y_encoder, preds_idx)
    report = classification_report(y_encoder, preds_idx, target_names=le.classes_)
    print("Accuracy:", acc)
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", report)
    return acc, cm, report

In [22]:
eval_sklearn(logistic_model, X_train, y_train, le)

Accuracy: 1.0
Confusion Matrix:
 [[12  0]
 [ 0 12]]
Classification Report:
               precision    recall  f1-score   support

        bear       1.00      1.00      1.00        12
      pandas       1.00      1.00      1.00        12

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



(1.0,
 array([[12,  0],
        [ 0, 12]]),
 '              precision    recall  f1-score   support\n\n        bear       1.00      1.00      1.00        12\n      pandas       1.00      1.00      1.00        12\n\n    accuracy                           1.00        24\n   macro avg       1.00      1.00      1.00        24\nweighted avg       1.00      1.00      1.00        24\n')

In [23]:
eval_sklearn(random_forest_model, X_train, y_train, le)

Accuracy: 1.0
Confusion Matrix:
 [[12  0]
 [ 0 12]]
Classification Report:
               precision    recall  f1-score   support

        bear       1.00      1.00      1.00        12
      pandas       1.00      1.00      1.00        12

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



(1.0,
 array([[12,  0],
        [ 0, 12]]),
 '              precision    recall  f1-score   support\n\n        bear       1.00      1.00      1.00        12\n      pandas       1.00      1.00      1.00        12\n\n    accuracy                           1.00        24\n   macro avg       1.00      1.00      1.00        24\nweighted avg       1.00      1.00      1.00        24\n')

In [24]:
eval_cnn(cnn_model, X, y_encoder, le)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Accuracy: 0.5
Confusion Matrix:
 [[ 0 15]
 [ 0 15]]
Classification Report:
               precision    recall  f1-score   support

        bear       0.00      0.00      0.00        15
      pandas       0.50      1.00      0.67        15

    accuracy                           0.50        30
   macro avg       0.25      0.50      0.33        30
weighted avg       0.25      0.50      0.33        30



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.5,
 array([[ 0, 15],
        [ 0, 15]]),
 '              precision    recall  f1-score   support\n\n        bear       0.00      0.00      0.00        15\n      pandas       0.50      1.00      0.67        15\n\n    accuracy                           0.50        30\n   macro avg       0.25      0.50      0.33        30\nweighted avg       0.25      0.50      0.33        30\n')

In [25]:
# Cell 11: Predict on a single image using all models
import joblib
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

def prepare_single_image(path_or_pil, img_size=(224,224)):
    if isinstance(path_or_pil, str):
        img = Image.open(path_or_pil).convert("RGB")
    else:
        img = path_or_pil.convert("RGB")
    img = img.resize(img_size)
    arr = np.array(img)
    arr_flat = arr.reshape(1, -1)
    arr_cnn = arr.reshape(1, img_size[0], img_size[1], 3).astype('float32')/255.0
    return arr_flat, arr_cnn, arr, img

def predict_all_models(image_path, le):
    # load models
    log = joblib.load("models/logistic.pkl")
    rf = joblib.load("models/randomforest.pkl")
    cnn = load_model("models/cnn.keras")

    arr_flat, arr_cnn, arr_raw, pil_img = prepare_single_image(image_path)

    p_log = log.predict(arr_flat)[0]
    p_rf = rf.predict(arr_flat)[0]
    p_cnn_idx = np.argmax(cnn.predict(arr_cnn)[0])
    p_cnn = le.inverse_transform([p_cnn_idx])[0]

    print("Logistic:", le.inverse_transform([p_log])[0] if isinstance(p_log, (int, np.integer)) else p_log)
    print("Random Forest:", le.inverse_transform([p_rf])[0] if isinstance(p_rf, (int, np.integer)) else p_rf)
    print("CNN:", p_cnn)


In [26]:
predict_all_models('pandas1.jpeg', le)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Logistic: pandas
Random Forest: pandas
CNN: pandas


In [27]:
predict_all_models('bear1.jpeg', le)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Logistic: pandas
Random Forest: pandas
CNN: pandas
